# Importing data & modules

In [1]:
from tensorflow.keras import datasets
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
#load data
(X_train, y_train), (X_test, y_test)=datasets.cifar10.load_data()

X_train.shape

(50000, 32, 32, 3)

In [3]:
# resize the images into (224, 224) pixels
X_train = tf.image.resize(X_train[:1000], (224, 224))

X_test = tf.image.resize(X_test[:1000], (224, 224))

print("X_train224 : ", X_train.shape)
print("X_train224 : ", X_test.shape)

X_train224 :  (1000, 224, 224, 3)
X_train224 :  (1000, 224, 224, 3)


In [4]:
## Transforming labels to correct format
y_train = to_categorical(y_train[:1000], num_classes=10)
y_test = to_categorical(y_test[:1000], num_classes=10)

print("y_train : ", y_train.shape)
print("y_test : ", y_test.shape)

y_train :  (1000, 10)
y_test :  (1000, 10)


# VGG 16

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
# We use Include_top=False to remove the classification layer that was trained on the ImageNet dataset 
# and set the model as not trainable.
base_model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
base_model.trainable = False ## Not trainable weights

In [6]:
## here i used the preprocess_input function from VGG16 to normalize the input data.
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [7]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(10, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [8]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

history = model.fit(X_train, y_train, epochs=20, 
                    validation_data=(X_test, y_test))

Epoch 1/20
32/32 [==============================] - 658s 21s/step - loss: 2.6133 - accuracy: 0.2430 - val_loss: 1.8923 - val_accuracy: 0.3220
Epoch 2/20
32/32 [==============================] - 661s 21s/step - loss: 1.6455 - accuracy: 0.4540 - val_loss: 1.7906 - val_accuracy: 0.3690
Epoch 3/20
32/32 [==============================] - 681s 22s/step - loss: 1.2447 - accuracy: 0.5820 - val_loss: 1.7977 - val_accuracy: 0.4650
Epoch 4/20
25/32 [======================>.......] - ETA: 1:16 - loss: 0.9607 - accuracy: 0.6538

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\kaddi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kaddi\AppData\Local\Temp/ipykernel_11072/1084869378.py", line 7, in <module>
    history = model.fit(X_train, y_train, epochs=20,
  File "C:\Users\kaddi\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\kaddi\Anaconda3\lib\site-packages\keras\engine\training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\kaddi\Anaconda3\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\kaddi\Anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\kaddi\Anaconda3\lib\site-packages\tensorflow\pyth

TypeError: object of type 'NoneType' has no len()

In [ ]:
import matplotlib.pyplot as plt
def plot_accuracy_functions(history):    
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')
    
def plot_loss_functions(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
plot_accuracy_functions(history)

In [ ]:
plot_loss_functions(history)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size= 32, verbose=2)

# ResNet 50

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, models, layers, optimizers
import tensorflow as tf

from tensorflow.keras.layers import Dense, Input, MaxPooling2D, ZeroPadding2D, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from tensorflow.keras.layers import Input
from tensorflow.keras import datasets, models, layers, optimizers

In [ ]:
def identity_block(X, f, filters, training=True, initializer=glorot_uniform):
    
    # Retrieve Filters
    F1, F2, F3 = filters
    X_shortcut = X
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)
    
    ## Second component of main path
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X) 

    ## Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X,X_shortcut])
    X =  Activation('relu')(X) 

    return X

In [ ]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):

    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape = (224, 224, 1), classes = 17):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    ## Stage 3 
    X = convolutional_block(X, f=3, filters = [128,128,512], s = 2) 
    X = identity_block(X, 3, [128, 128, 512]) 
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    
    ## Stage 4 
    X = convolutional_block(X, f=3, filters = [256,256,1024], s = 2) 
    X = identity_block(X, 3, [256, 256, 1024]) 
    X = identity_block(X, 3, [256, 256, 1024])  
    X = identity_block(X, 3, [256, 256, 1024])  
    X = identity_block(X, 3, [256, 256, 1024]) 
    X = identity_block(X, 3, [256, 256, 1024]) 

    ## Stage 5 
    X = convolutional_block(X, f=3, filters = [512,512,2048], s = 2)  
    X = identity_block(X, 3, [512, 512, 2048])  
    X = identity_block(X, 3, [512, 512, 2048]) 

    ## AVGPOOL
    X = AveragePooling2D((2,2))(X) 
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = models.Model(inputs = X_input, outputs = X)

    return model

In [ ]:
#Create and Compile the model
ResNet_model = ResNet50(input_shape = (224, 224, 3), include_top=False, classes = 10)
ResNet_model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ResNet_history = ResNet_model.fit(X_train, y_train, epochs =2, batch_size=32, shuffle=True,
        validation_data=(X_test, y_test))